# 1. 모델 선정
pycaret 패키지 내 AutoML을 적용해 데이터셋에 가장 적합한 모델을 선정한다.

데이터 불러오기
전처리와 변수 선정이 완료된 데이터를 불러온다.

In [1]:
import re
import os

import pandas as pd
import numpy as np
import pycaret    

import sys, gc, warnings, datetime


In [2]:
pycaret.__version__

'3.0.0'

In [3]:
%pwd

'/Users/a06411/Documents/GitHub/js_automl_pycaret/lending_club_da/Modeling'

In [4]:
path_read = '/Users/a06411/Documents/data_hub/lending_club/lgb_selected.pkl'

In [5]:
df = pd.read_pickle(path_read )

In [6]:
df.shape

(1382351, 63)

In [7]:
df_sample = df.copy()

In [10]:
df_sample = df_sample.sample(frac=0.01, replace=True, random_state=2020)

In [11]:
df_sample.shape

(13824, 63)

In [12]:
df_sample.columns

Index(['int_rate', 'dti', 'annual_inc', 'mo_sin_old_rev_tl_op',
       'acc_open_past_24mths', 'loan_amnt', 'emp_length', 'addr_state',
       'revol_bal', 'term', 'sub_grade', 'funded_amnt_inv', 'installment',
       'purpose', 'total_rev_hi_lim', 'fico_range_low', 'debt_settlement_flag',
       'mort_acc', 'total_bc_limit', 'home_ownership', 'avg_cur_bal',
       'all_util', 'mths_since_recent_bc', 'total_acc', 'open_acc_6m',
       'bc_util', 'num_actv_rev_tl', 'funded_amnt', 'hardship_flag',
       'num_rev_tl_bal_gt_0', 'mths_since_recent_inq', 'inq_last_6mths',
       'num_il_tl', 'mo_sin_old_il_acct', 'num_rev_accts', 'num_tl_120dpd_2m',
       'total_il_high_credit_limit', 'application_type', 'revol_util',
       'tot_hi_cred_lim', 'delinq_2yrs', 'mo_sin_rcnt_tl', 'num_actv_bc_tl',
       'mths_since_last_record', 'percent_bc_gt_75', 'bc_open_to_buy',
       'max_bal_bc', 'grade', 'open_rv_24m', 'mo_sin_rcnt_rev_tl_op',
       'pct_tl_nvr_dlq', 'verification_status', 'tot_cur_b

In [13]:
from pycaret.classification import *

In [15]:
cols = [*df_sample.columns[:-2]]

In [16]:
data = df_sample[cols]

In [19]:
clf = setup(data = data, target = 'loan_status')

,Description,Value
0,Session id,664
1,Target,loan_status
2,Target type,Binary
3,Original data shape,"(13980, 61)"
4,Transformed data shape,"(14916, 61)"
5,Transformed train set shape,"(10467, 61)"
6,Transformed test set shape,"(4449, 61)"
7,Numeric features,60
8,Preprocess,True
9,Imputation type,simple


In [20]:
best_3 = compare_models(sort ='AUC', fold=3, n_select = 3)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
et,Extra Trees Classifier,0.8282,0.7742,0.2156,0.8610,0.3446,0.2846,0.3764,0.4200
catboost,CatBoost Classifier,0.8268,0.7706,0.2521,0.7628,0.3787,0.3066,0.3707,1.6667
rf,Random Forest Classifier,0.8298,0.7702,0.2252,0.8599,0.3566,0.2954,0.3847,0.4467
lightgbm,Light Gradient Boosting Machine,0.8213,0.7613,0.2657,0.6937,0.3840,0.3030,0.3511,0.7233
gbc,Gradient Boosting Classifier,0.8256,0.7611,0.2393,0.7710,0.3649,0.2950,0.3636,1.5300
lda,Linear Discriminant Analysis,0.8193,0.7451,0.2065,0.7531,0.3234,0.2565,0.3291,0.2367
xgboost,Extreme Gradient Boosting,0.8157,0.7412,0.2958,0.6288,0.4020,0.3095,0.3404,0.8700
ada,Ada Boost Classifier,0.8184,0.7377,0.2357,0.6984,0.3520,0.2755,0.3310,0.7200
qda,Quadratic Discriminant Analysis,0.8138,0.7196,0.1190,0.9406,0.2112,0.1723,0.2964,0.2167
nb,Naive Bayes,0.6002,0.6683,0.6732,0.2988,0.4138,0.1740,0.2072,0.6267


AUC 기준 Catboost, XGBoost, LightGBM이 우수한 모델로 선정되었다.
3개 모델을 대상으로 파라미터 최적화를 진행한다.

In [21]:
# train a decision tree model
dt = create_model('dt')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7650,0.6516,0.4566,0.4405,0.4484,0.2992,0.2993
1,0.7287,0.6169,0.4247,0.3705,0.3957,0.2219,0.2228
2,0.7507,0.6191,0.3927,0.4019,0.3972,0.2401,0.2401
3,0.7507,0.6420,0.4545,0.4149,0.4338,0.2744,0.2749
4,0.7364,0.6380,0.4682,0.3931,0.4274,0.2579,0.2595
5,0.7431,0.6138,0.3909,0.3891,0.3900,0.2273,0.2273
6,0.7469,0.6213,0.4045,0.3991,0.4018,0.2413,0.2413
7,0.7304,0.6079,0.3973,0.3671,0.3816,0.2096,0.2098
8,0.7409,0.6297,0.4384,0.3934,0.4147,0.2489,0.2495


In [22]:
# tune hyperparameters of decision tree
tuned_dt = tune_model(dt)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8118,0.7316,0.2192,0.6486,0.3276,0.2482,0.2980
1,0.8157,0.7160,0.1370,0.8824,0.2372,0.1917,0.3032
2,0.8109,0.7217,0.1233,0.8182,0.2143,0.1687,0.2701
3,0.8214,0.7448,0.1591,0.9459,0.2724,0.2255,0.3457
4,0.8290,0.7335,0.1955,0.9556,0.3245,0.2726,0.3878
5,0.8204,0.7189,0.2045,0.7759,0.3237,0.2587,0.3363
6,0.8157,0.7387,0.1591,0.8140,0.2662,0.2120,0.3067
7,0.8107,0.6936,0.2146,0.6438,0.3219,0.2426,0.2925
8,0.8193,0.7111,0.1370,1.0000,0.2410,0.2006,0.3339


Fitting 10 folds for each of 10 candidates, totalling 100 fits


In [23]:
# tune hyperparameters with increased n_iter
tuned_dt1 = tune_model(dt, n_iter = 50)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8290,0.7419,0.2694,0.7564,0.3973,0.3229,0.3817
1,0.8281,0.7227,0.2740,0.7407,0.4000,0.3236,0.3785
2,0.8090,0.7292,0.1963,0.6418,0.3007,0.2247,0.2781
3,0.8262,0.7316,0.2727,0.7317,0.3974,0.3197,0.3732
4,0.8243,0.7432,0.2682,0.7195,0.3907,0.3123,0.3645
5,0.8052,0.7136,0.2000,0.6111,0.3014,0.2206,0.2675
6,0.8271,0.7324,0.2545,0.7671,0.3823,0.3100,0.3743
7,0.8212,0.7179,0.2329,0.7286,0.3529,0.2799,0.3418
8,0.8317,0.7042,0.2557,0.8116,0.3889,0.3207,0.3934


Fitting 10 folds for each of 50 candidates, totalling 500 fits


In [24]:
# tune hyperparameters to optimize AUC
tuned_dt2 = tune_model(dt, optimize = 'AUC') #default is 'Accuracy'

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8118,0.7316,0.2192,0.6486,0.3276,0.2482,0.2980
1,0.8157,0.7160,0.1370,0.8824,0.2372,0.1917,0.3032
2,0.8109,0.7217,0.1233,0.8182,0.2143,0.1687,0.2701
3,0.8214,0.7448,0.1591,0.9459,0.2724,0.2255,0.3457
4,0.8290,0.7335,0.1955,0.9556,0.3245,0.2726,0.3878
5,0.8204,0.7189,0.2045,0.7759,0.3237,0.2587,0.3363
6,0.8157,0.7387,0.1591,0.8140,0.2662,0.2120,0.3067
7,0.8107,0.6936,0.2146,0.6438,0.3219,0.2426,0.2925
8,0.8193,0.7111,0.1370,1.0000,0.2410,0.2006,0.3339


Fitting 10 folds for each of 10 candidates, totalling 100 fits


In [25]:
# tune hyperparameters with custom_grid
params = {"max_depth": np.random.randint(1, (len(data.columns)*.85),20),
          "max_features": np.random.randint(1, len(data.columns),20),
          "min_samples_leaf": [2,3,4,5,6],
          "criterion": ["gini", "entropy"]
          }

tuned_dt_custom = tune_model(dt, custom_grid = params)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8118,0.7445,0.2466,0.6279,0.3541,0.2677,0.3080
1,0.8147,0.7336,0.2329,0.6623,0.3446,0.2646,0.3139
2,0.8138,0.7251,0.1553,0.7727,0.2586,0.2028,0.2902
3,0.8243,0.7292,0.2318,0.7727,0.3566,0.2875,0.3582
4,0.8176,0.7270,0.2455,0.6835,0.3612,0.2814,0.3320
5,0.8176,0.7130,0.2136,0.7231,0.3298,0.2588,0.3239
6,0.8166,0.7465,0.2045,0.7258,0.3191,0.2498,0.3176
7,0.8155,0.6952,0.2420,0.6625,0.3545,0.2731,0.3205
8,0.8270,0.7081,0.2283,0.8065,0.3559,0.2903,0.3684


Fitting 10 folds for each of 10 candidates, totalling 100 fits


# 하이퍼파라미터 최적화: 그리드서치(Grid Search)
아래와 같은 과정으로 그리드서치 하이퍼파라미터 최적화를 진행한다.

과거 데이터가 학습된 모델을 통해 현 시점의 대출 사기 여부를 예측하는 현업 방식과 유사하도록 아래와 같은 데이터셋을 구성한다.

최적의 하이퍼파라미터 탐색: 학습(2017년 이전 데이터) / 평가(2017년 데이터)

In [26]:
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold, GridSearchCV

In [43]:
path_read1 = '/Users/a06411/Documents/data_hub/lending_club/feature_selected_ld_1.pkl'

In [44]:
df_f = pd.read_pickle(path_read1)

In [46]:
df_f.shape

(1382351, 61)

In [47]:
df_f.columns

Index(['int_rate', 'dti', 'annual_inc', 'mo_sin_old_rev_tl_op',
       'acc_open_past_24mths', 'loan_amnt', 'emp_length', 'revol_bal', 'term',
       'funded_amnt_inv', 'installment', 'purpose', 'total_rev_hi_lim',
       'fico_range_low', 'debt_settlement_flag', 'mort_acc', 'total_bc_limit',
       'home_ownership', 'avg_cur_bal', 'all_util', 'mths_since_recent_bc',
       'total_acc', 'open_acc_6m', 'bc_util', 'num_actv_rev_tl', 'funded_amnt',
       'hardship_flag', 'num_rev_tl_bal_gt_0', 'mths_since_recent_inq',
       'inq_last_6mths', 'num_il_tl', 'mo_sin_old_il_acct', 'num_rev_accts',
       'num_tl_120dpd_2m', 'total_il_high_credit_limit', 'application_type',
       'revol_util', 'tot_hi_cred_lim', 'delinq_2yrs', 'mo_sin_rcnt_tl',
       'num_actv_bc_tl', 'mths_since_last_record', 'percent_bc_gt_75',
       'bc_open_to_buy', 'max_bal_bc', 'grade', 'open_rv_24m',
       'mo_sin_rcnt_rev_tl_op', 'pct_tl_nvr_dlq', 'verification_status',
       'tot_cur_bal', 'total_bal_ex_mort', '

## 모델 성능 평가용 데이터
최적화된 하이퍼파라미터를 적용해 최종 모델 성능 지표를 산출할 데이터를 분리한다.

학습(2007~2018) / 평가(2018)
대출 신청 연도 추출:

In [48]:
df_f['issue_d'] = pd.to_datetime(df_f['issue_d'])
df_f['issue_year'] = df_f['issue_d'].dt.year

In [49]:
df_f.shape

(1382351, 62)

In [50]:
train = df_f[df_f['issue_year'] != 2018]
test = df_f[df_f['issue_year'] == 2018]

train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

answer = test['loan_status']

In [51]:
train.shape

(1314290, 62)

In [52]:
test.shape

(68061, 62)

In [53]:
X = train.copy()
y = train['loan_status']

X.drop(['loan_status'], axis = 1, inplace = True)

print(X.shape)
print(y.shape)

(1314290, 61)
(1314290,)


# 하이퍼파라미터 탐색용 데이터
그리드서치(Grid Search)기법으로 하이퍼파라미터 최적화를 진행할 데이터를 분리한다.

학습(2007~2017) / 검증(2017)
학습/검증 데이터 분리:

In [54]:
ttrain = train[train['issue_year'] != 2017]
ttest = train[train['issue_year'] == 2017]

In [55]:
ttrain.shape, ttest.shape

((1132562, 62), (181728, 62))

## 데이터 내 독립변수(X_train, X_test) / 종속변수(y_train, y_test) 분리:

In [37]:
X_train = ttrain.copy()
y_train = ttrain['loan_status']
X_train.drop(['loan_status'], axis = 1, inplace = True)

X_test = ttest.copy()
y_test = ttest['loan_status']
X_test.drop(['loan_status'], axis = 1, inplace = True)

In [56]:
remove_features = ['earliest_cr_line', 'issue_d', 'last_credit_pull_d', 
                   'last_pymnt_d', 'next_pymnt_d',
                  'initial_list_status','out_prncp','out_prncp_inv',
                   'total_pymnt','total_pymnt_inv','total_rec_prncp',
                   'total_rec_int','total_rec_late_fee',
                   'recoveries','collection_recovery_fee','last_pymnt_amnt',
                   'last_fico_range_high','last_fico_range_low' , 
                   'id', 'loan_status' , 'issue_year',
                   'addr_state','sub_grade' ]
features  = [col for col in list(train) if col not in remove_features]

## 하이퍼파라미터 최적화(Grid Search)
아래와 같은 과정으로 그리드서치 하이퍼파라미터 최적화를 진행한다.

하이퍼파라미터 초기값 지정
하이퍼파라미터 별 탐색 범위 지정
그리드 서치 후 최적값 저장
하이퍼파라미터 초기값 지정:

In [57]:
params = {
    'application': 'binary',
    'boosting': 'gbdt', 
    'num_iterations': 100, 
    'learning_rate': 0.05,
    'num_leaves': 62,
    'max_depth': -1,
    'max_bin': 510, 
    'lambda_l1': 5, 
    'lambda_l2': 10, 
    'metric' : 'binary_error',
    'subsample_for_bin': 200,
    'subsample': 1,
    'colsample_bytree': 0.8, 
    'min_split_gain': 0.5, 
    'min_child_weight': 1, 
    'min_child_samples': 5
}

mdl = lgb.LGBMClassifier(boosting_type= 'gbdt', 
          objective = 'binary', 
          n_jobs = 5, 
          max_depth = params['max_depth'],
          max_bin = params['max_bin'], 
          subsample_for_bin = params['subsample_for_bin'],
          subsample = params['subsample'], 
          min_split_gain = params['min_split_gain'], 
          min_child_weight = params['min_child_weight'], 
          min_child_samples = params['min_child_samples'])

mdl.get_params().keys()

dict_keys(['boosting_type', 'class_weight', 'colsample_bytree', 'importance_type', 'learning_rate', 'max_depth', 'min_child_samples', 'min_child_weight', 'min_split_gain', 'n_estimators', 'n_jobs', 'num_leaves', 'objective', 'random_state', 'reg_alpha', 'reg_lambda', 'subsample', 'subsample_for_bin', 'subsample_freq', 'silent', 'max_bin'])

## 하이퍼파라미터 별 탐색 범위 지정:

- bagging_fraction: 데이터의 일부만 사용하는 bagging의 비율
- learning_rate: 학습 gradient에 부여하는 가중치
- num_leaves: 예측 트리에서 데이터가 분기되는 가지의 수
- max_depth: 에측 트리의 최대 깊이
- reg_alpha: L1 정규화 비율

In [58]:
gridParams = {
    'bagging_fraction': [0.6, 0.8],
    'learning_rate': [0.01, 0.1],
    'num_leaves': [125, 255],
    'max_depth': [10,20],
    'reg_alpha' : [0.5,1]
    }

## 그리드 서치:

- 학습 데이터: X_train, y_train(2007~2017)
- 검증 데이터: X_test, y_test(2017)

In [59]:
gridParams = {
    'bagging_fraction': [0.6, 0.8],
    'learning_rate': [0.01, 0.1],
    'num_leaves': [125, 255],
    'max_depth': [10,20],
    'reg_alpha' : [0.5,1]
    }

grid = GridSearchCV(mdl, gridParams, verbose=1, cv=4, n_jobs=-1)

grid.fit(X_train[features], y_train, eval_metric='auc',
         eval_set=[(X_train[features], y_train), (X_test[features], y_test)])

print(grid.best_params_)
print(grid.best_score_)

Fitting 4 folds for each of 32 candidates, totalling 128 fits
[LightGBM] [Warning] Unknown parameter: silent
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1 will be ignored. Current value: bagging_fraction=0.6
[LightGBM] [Warning] Using too small ``bin_construct_sample_cnt`` may encounter unexpected errors and poor accuracy.
[LightGBM] [Warning] Unknown parameter: silent
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1 will be ignored. Current value: bagging_fraction=0.6
[LightGBM] [Info] Number of positive: 171610, number of negative: 677812
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.191450 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1692
[LightGBM] [Info] Number of data points in the train set: 849422, number of used features: 56
[LightGBM] [Warning] Unknown parameter: silent
[LightGBM] [Warning]

In [61]:
print(grid.best_params_)
print(grid.best_score_)

{'bagging_fraction': 0.6, 'learning_rate': 0.1, 'max_depth': 10, 'num_leaves': 125, 'reg_alpha': 0.5}
0.8261101735133024


## 최적의 파라미터 저장

In [62]:
params['bagging_fraction'] = grid.best_params_['bagging_fraction']
params['learning_rate'] = grid.best_params_['learning_rate']
params['num_leaves'] = grid.best_params_['num_leaves']
params['max_depth'] = grid.best_params_['max_depth']
params['reg_alpha'] = grid.best_params_['reg_alpha']

In [64]:
def seed_everything(seed=0):
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
SEED = 42
seed_everything(SEED)

# 예측 및 성능 평가 함수:

k-fold 교차검증을 통한 과적합 이슈 방지
하이퍼파라미터 최적값 지정

In [71]:
def make_lgb_prediction(train, y, test, features, categorical_features='auto', model_params=None, folds=5):
    
    skf = KFold(n_splits=folds, random_state=SEED, shuffle=True)
    folds = 5
    x_train = train[features]
    x_test = test[features]

    y_preds = np.zeros(x_test.shape[0])
    y_oof = np.zeros(x_train.shape[0])
    score = 0    

    for fold, (tr_idx, val_idx) in enumerate(skf.split(x_train, y)):
        print(f'Fold: {fold+1}')

        x_tr, x_val = x_train.loc[tr_idx, features], x_train.loc[val_idx, features]
        y_tr, y_val = y[tr_idx], y[val_idx]

        print(x_tr.shape, x_val.shape)

        dtrain = lgb.Dataset(x_tr, label=y_tr)
        dvalid = lgb.Dataset(x_val, label=y_val)

        clf = lgb.train(   
            model_params,
            dtrain,
            valid_sets=[dtrain, dvalid],
            categorical_feature=categorical_features,
        )
        
        y_pred_val = clf.predict(x_val)

        y_oof[val_idx] = y_pred_val
        print(f"Fold {fold + 1} | AUC Score: {roc_auc_score(y_val, y_pred_val)}")

        score += roc_auc_score(y_val, y_pred_val) / folds
        y_preds += clf.predict(x_test) / folds

        del x_tr, x_val, y_tr, y_val
        gc.collect()

    print(f"\nMean AUC score = {score}")
    print(f"OOF AUC score = {roc_auc_score(y, y_oof)}")
    
    return y_oof, y_preds, clf

## 모델 성능 확인:

학습 데이터: X, y(2007~2018)
평가 데이터: test(2018)


In [72]:
y_oof_lgb, y_preds_lgb, mdl = make_lgb_prediction(X, y, test, features, model_params=params)

Fold: 1
(1051432, 58) (262858, 58)
[LightGBM] [Warning] lambda_l1 is set=5, reg_alpha=0.5 will be ignored. Current value: lambda_l1=5
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1 will be ignored. Current value: bagging_fraction=0.6
[LightGBM] [Warning] Using too small ``bin_construct_sample_cnt`` may encounter unexpected errors and poor accuracy.
[LightGBM] [Warning] lambda_l1 is set=5, reg_alpha=0.5 will be ignored. Current value: lambda_l1=5
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1 will be ignored. Current value: bagging_fraction=0.6
[LightGBM] [Info] Number of positive: 222028, number of negative: 829404
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.047769 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1783
[LightGBM] [Info] Number of data points in the train set: 1051432, number of used fe

NameError: name 'gc' is not defined